# **Text Vectorization II**

---
### Natural Language Processing
Date: Nov 16, 2022

Author: Lorena Calvo-Bartolomé (lcalvo@pa.uc3m.es)

Version 1.0

---
This notebook is based on the Gensim documentation and tutorials, as well as on Daniel Voigt Godoy's book "Deep Learning with Pytorch Step-by-Step, Volume III: Sequences \& NLP".

Our goal here is to provide a basic overview of the following text vectorization techniques:


*   Word2Vec
*   GloVe
*   FastText
*   ELMo

as well as how to use them to solve a Text Classification task.

In [1]:
%pip install gensim==4.2.0 pqkmeans

     ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
     - -------------------------------------- 1.0/24.0 MB 22.0 MB/s eta 0:00:02
     - -------------------------------------- 1.0/24.0 MB 22.0 MB/s eta 0:00:02
     - -------------------------------------- 1.0/24.0 MB 22.0 MB/s eta 0:00:02
     --- ------------------------------------ 2.1/24.0 MB 12.1 MB/s eta 0:00:02
     --- ------------------------------------ 2.1/24.0 MB 12.1 MB/s eta 0:00:02
     ----- ---------------------------------- 3.1/24.0 MB 11.2 MB/s eta 0:00:02
     ----- ---------------------------------- 3.1/24.0 MB 11.2 MB/s eta 0:00:02
     ----- ---------------------------------- 3.1/24.0 MB 11.2 MB/s eta 0:00:02
     ------ --------------------------------- 4.2/24.0 MB 10.7 MB/s eta 0:00:02
     ------ --------------------------------- 4.2/24.0 MB 10.7 MB/s eta 0:00:02
     ------ --------------------------------- 4.2/24.0 MB 10.7 MB/s eta 0:00:02
     ------ --------------------------------- 4.

  error: subprocess-exited-with-error
  
  × Building wheel for pqkmeans (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [92 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-38
      creating build\lib.win-amd64-cpython-38\pqkmeans
      copying pqkmeans\evaluation.py -> build\lib.win-amd64-cpython-38\pqkmeans
      copying pqkmeans\__init__.py -> build\lib.win-amd64-cpython-38\pqkmeans
      creating build\lib.win-amd64-cpython-38\test
      copying test\__init__.py -> build\lib.win-amd64-cpython-38\test
      creating build\lib.win-amd64-cpython-38\pqkmeans\clustering
      copying pqkmeans\clustering\bkmeans.py -> build\lib.win-amd64-cpython-38\pqkmeans\clustering
      copying pqkmeans\clustering\cpp_implemented_clustering_sample.py -> build\lib.win-amd64-cpython-38\pqkmeans\clustering
      copying pqkmeans\clustering\pqkmeans.py -> build\lib.win-amd64-cpython-38\

In [2]:
# Common imports
import os
import numpy as np
import pandas as pd
from termcolor import colored
import seaborn as sns
import matplotlib.pyplot as plt
import tqdm
import scipy
import gc


# Figures plotted inside the notebook
%matplotlib inline
# High quality figures
%config InlineBackend.figure_format = 'retina'
# Figures style
plt.style.use('seaborn-whitegrid')
sns.set_style("darkgrid")
sns.color_palette("deep")
# Figues size
plt.rcParams['figure.figsize'] = [8, 6]

# Supress future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings(action='ignore',module='compress_fasttext')

C:\Users\Horiz\AppData\Local\Temp\ipykernel_27592\4241668263.py:18: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


In [3]:
pip show pandas

Name: pandas
Version: 2.0.3
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: 
Author: 
Author-email: The Pandas Development Team <pandas-dev@python.org>
License: BSD 3-Clause License
        
        Copyright (c) 2008-2011, AQR Capital Management, LLC, Lambda Foundry, Inc. and PyData Development Team
        All rights reserved.
        
        Copyright (c) 2011-2023, Open source contributors.
        
        Redistribution and use in source and binary forms, with or without
        modification, are permitted provided that the following conditions are met:
        
        * Redistributions of source code must retain the above copyright notice, this
          list of conditions and the following disclaimer.
        
        * Redistributions in binary form must reproduce the above copyright notice,
          this list of conditions and the following disclaimer in the documentation
          and/or other materials provided with the distrib

In [4]:
# To wrap long text lines
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
# For fancy table Display
%load_ext google.colab.data_table

In [ ]:
path_to_folder = '/content/drive/My Drive/NLP_IA'  # UPDATE THIS ACCORDING TO WHERE YOU WANT TO SAVE THE FILES!!!!

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

# Change to assignment directory
os.chdir(path_to_folder)

## **1. Data preparation**
---


For this second text vectorization notebook, we will keep working with the **IMDB Dataset of 50K Movie Reviews**. Since we already carried out all the dataset preparation in the previous notebook, we will just load the the dataframe that we saved.

In [8]:
import pickle

def unpickler(file: str):
    """Unpickle file"""
    with open(file, 'rb') as f:
        return pd.compat.pickle_compat.load(f)

corpus_df = unpickler("corpus_df_imbdb.pickle")
print(len(corpus_df))

15000


##### **Exercise 1**

Create a Gensim corpus and obtain its Gensim dictionary. With the latter, save the vocabulary of the corpus as a list. Store the Gensim corpus in a variable named ``corpus``, the Gensim dictionary in a variable named ``D``, and the vocabulary in a variable named ``vocab``.

**Hint:** Make use of the Gensim dictionary' method ``token2id`` for obtaining the vocabulary list.

In [ ]:
# <SOL>

# </SOL>

print(colored('\n============= First 10 words in the vocabulary =============', 'blue'))
print(vocab[0:20])

Let's also save the sentiment of the review in a NumPy array for later use.

In [ ]:
Y = corpus_df['binary_sentiment'].values

## **2. Word2Vec**
---


We will use [Gensim's implementation of Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html), which supports both the skip-gram and CBOW models, with either hierarchical softmax or negative sampling. Furthermore, Gensim contains a collection of functions for the exploitation of pre-trained embeddings, which can be either embeddings developed by us or pre-trained models on big corpora.

### *2.1. Implementation of Word2Vec in Gensim*

Word2Vec models require a lot of text, e.g., the entire Wikipedia corpus. Nevertheless, we will demonstrate the principles using a small example from our IMDB dataset.

Gensim's Word2Vec expects a sequence of sentences (``sentences``) as its input where each sentence is a list of words:

```
sentences = [['This', 'is', 'the', 'first', 'sentence'], ['This', 'is', 'the', 'second', 'sentence']]
```

In addition, some of the most important parameters that we may wish to configure are the following:

* `vector_size`: The dimensionality of the word vectors, i.e.,  the embedding size. The default is **100**.
* `window`: The window size, i.e., the number of adjacent words that are considered in the same context as the target word. The default is **5**.
* `min_count`: Ignores all words with a total frequency lower than this.
* `sg`: Training algorithm: 1 for skip-gram; otherwise CBOW.

Keeping the input as a Python built-in list is convenient, but can use up a lot of RAM when the input is large. Gensim only requires the input to provide sentences sequentially, when iterated over. Hence, we can use the same principle that we utilized in the first notebook of this tutorial at ``2.5. Memory efficient computation``.

##### **Exercise 2**

The provided class ``IterableSentence_fromfile`` creates an iterator that yields one sentence (list of utf8 words) after another from the file ``"imdb_lemmas_clean.txt"``.

Use the sentence iterator to create a Word2Vec skip-gram model named ``model_w2v`` with vector size **200** and window size **5**.  Words must appear in at least **10** documents to be kept in the model. Use a seed of **42**.

In [ ]:
%%time
from gensim.models import Word2Vec

class IterableSentence_fromfile(object):
    def __init__(self, filename):
        self.__filename = filename

    def __iter__(self):
        for line in open(self.__filename):
            # assume there's one sentence per line, tokens separated by whitespace
            yield line.split()

sentences = IterableSentence_fromfile("imdb_lemmas_clean.txt") # a memory-friendly iterator

# <SOL>

# </SOL>

Once the model is trained, it is accessible via the ``wv`` attribute, which is the actual word vector model in which queries can be made.

We can see the learned vocabulary of the model through the dictionary ``key_to_index``, that is, the target words for which we have calculated an embedding, as well as the corresponding embeddings for these words, as follows:

In [ ]:
print(colored('\n============= Word2Vec vocabulary =============', 'blue'))
words = list(model_w2v.wv.key_to_index)
print(words[0:50])
print(colored('\n============= Embedding of the first word =============', 'blue'))
print(model_w2v.wv[words[0]])

We can save the trained model by calling the ``save()`` function, and we can load it again by calling the ``load()`` method.

In [ ]:
model_w2v.save("model_w2v_imbd_sampling03.model")

# This function will load the model, but we will not be using it here
#model_w2v = Word2Vec.load("model_w2v_imbd_sampling03.model")

Since we are not going to keep with the training at this moment, we do not need the full model state anymore. Hence, we can store the words and their trained embeddings and load them back as ``KeyedVectors``, which is essentially a mapping between keys and vectors. This results in a much smaller and faster object that can be mapped for lightning-fast loading and sharing the vectors in RAM between processes.

In [ ]:
from gensim.models import KeyedVectors

# Store just the words + their trained embeddings.
word_vectors = model_w2v.wv
word_vectors.save("model_w2v_imbd_sampling03.wordvectors")

# Load back with memory-mapping = read-only, shared across processes.
wv = KeyedVectors.load("model_w2v_imbd_sampling03.wordvectors", mmap='r')

Let's free the memory.



In [ ]:
del model_w2v
gc.collect()

Since we have represented our words as numerical vectors, we can utilize the Euclidean distance (or cosine similarity) among two word vectors to measure the linguistic or semantic similarity of the corresponding words. To do this, Gensim provides the function ``most_similar`` which returns the most similar words from a given word.

In [ ]:
wv.most_similar(positive="cinema")

In [ ]:
wv.most_similar(positive="restaurant")

We can also check the performance of our model by visualizing the embeddings. However, although embeddings are low-dimensional vectors, even 4 dimensions are too high to visualize. [t-SNE](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding) (*t-distributed Stochastic Neighbor Embedding*) solves this problem as it allows for visualizing high-dimensional data by reducing them to two or three-dimensional data. The technique takes in the embeddings (or any data) and aims at preserving as much as possible in 2 (or 3) dimensions the distances from the original space of the embedding dimension. This, therefore, helps us to get a feel for the space of word embedding.


Sklearn includes an implementation of [t-SNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) that we can easily use.

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(init='random')
embed_tsne = tsne.fit_transform(wv.vectors)

# We plot only 500 embeddings
fig, ax = plt.subplots(figsize=(16, 16))
for idx, word in enumerate((list(wv.key_to_index.keys())[:500])):
    plt.scatter(*embed_tsne[idx, :], color='steelblue')
    plt.annotate(word, (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)

plt.grid()

In [ ]:
del embed_tsne
gc.collect()

From the above results, we can see that our word2vec model does not do a bad job, but it could still be improved. Certainly, we need to train on a much larger dataset to obtain a coherent embedding structure.

### *2.2. Sentiment Analysis with ad-hoc trained Word2Vec embeddings*

Since our data contains reviews and not just words, we need to figure out a way to use the word vectors from the word2vec model to create a vector representation for an entire review. One simple solution to achieve this is to take the mean of all the word vectors present in the review.

Since when we constructed our Word2Vec model we filtered out those words which did not have an overall frequency of 10, we may encounter that our reviews contain some words that are not in the vocabulary of the model, i.e., they are out-of-vocabulary words (OOV).

##### **Exercise 3**

Implement the function ``get_vocabulary_coverage`` which, given a ``KeyedVectors`` object and the Gensim dictionary associated with the reviews, calculates the vocabulary coverage of the model. Take into account the frequency with which the OOV words appear in the reviews.

Then, calculate the coverage of the Word2Vec model on the IMDB vocabulary.

In [ ]:
def get_vocabulary_coverage(model, gensim_dict):
  # <SOL>
  # TODO: Implement body
  # </SOL>
  return coverage

# <SOL>
# TODO: Use function "get_vocabulary_coverage" to calculate coverage and print result
# </SOL>

##### **Exercise 4**

Implement the function ``get_review_vector`` which, given a ``KeyedVectors`` object, and a list of tokens representing a review returns the embedding vector for such a review by taking the average of the vectors of the words present in the given review. Ignore the OOV words.

Once you have the function, calculate the reviews' embedding vectors. Save them in a matrix of dimensions $[n_{reviews}, e_{size}]$, where $e_{size}$ is the embedding size of the model.

In [ ]:
def get_review_vector(model, review):
  # <SOL>
  # TODO: Implement body
  # </SOL>
  return vec

# <SOL>
# TODO: Calculate embedding vectors for each review and save them in a sparse matrix
# </SOL>

##### **Exercise 5**
Classify the reviews based on the Word2Vec embeddings representation with an SVM classifier. Split the data into a $70:30$ ratio for training and test with a $42$ random_state and use ``GridSearchCV`` to find the best hyperparameters values (``C`` and ``kernel``) for the classifier. Consider the grid search of the parameter ``C`` between 1 and 10 in intervals of 2, and try out the ``rbf`` and the ``linear`` kernels. At last, evaluate the performance of the classifier based on  its mean accuracy on the given test data and labels (``score`` function of the SVM classifier).

In [ ]:
# <SOL>

# </SOL>

## *3. GloVe*

### *3.1. Pre-trained GloVe models in Gensim*

For a specific NLP task, training our own word vectors may be the optimal strategy. However, it may require a long time, a fast computer with plenty of RAM and disc space, and maybe some experience in fine-tuning the input data and training algorithm. Another option is to just utilize pre-trained word embeddings.

For example, Gensim includes functions to exploit a set of [pre-trained *embeddings*](https://github.com/RaRe-Technologies/gensim-data).

In [ ]:
from gensim import downloader

print(colored('\n============= Available pre-trained models in Gensim =============', 'blue'))
print(list(downloader.info()['models'].keys()))

As we can see above, many of the pretrained models provided by Gensim are GloVe models. The pretrained GloVe embeddings come in many sizes and shapes, with dimensions in the range $[25,30]$, and vocabularies varying between $400,000$ and $2,200,000$ words.

Let's use Gensim's downloader to get the ``glove-wiki-gigaword-50``, which is the smallest available GloVe model. It was trained on Wikipedia 2014 and Gigawords, contains $400,000$ vocabulary words and its embeddings have a dimension of $50$.

In [ ]:
glove = downloader.load('glove-wiki-gigaword-50')

print(f"The dictionary size is {glove.vectors.shape[0]}.\n\
        Each embedding has a dimension of {glove.vectors.shape[1]}")

### *3.2. Semantic operations with embeddings*

One interesting thing that we can do with word vectors, which is easily perceptible with the GloVe-based embeddings we just loaded, is to perform algebra arithmetic with words. A famous example is the following equation:

``
queen = (king - man) + woman
``

That is, the word ``queen`` is the closest word to the operation consisting of subtracting ``man`` from ``king`` and adding the word ``woman``. The quality of "being a man" is replaced with that of "being a woman", leading to the word queen.

We can perform this operation in Gensim as follows:

In [ ]:
glove.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)

In [ ]:
synthetic_queen = glove['king'] - glove['man'] + glove['woman']
glove.similar_by_vector(synthetic_queen, topn=5)

The above result indicates that the term most comparable to the arithmetic operation result (synthetic queen) is the original word itself. This is pretty typical when conducting word arithmetic operations, which is why we usually omit the original word from the output, thus being the most similar word to our synthetic_queen the actual queen.

The general idea here is that the embeddings have learned to **encode abstract dimensions**, like gender, genealogy, or profession. Yet, none of these abstract dimensions correspond to a single numerical dimension. In its 50-dimensional feature space, the model learned to place ``the man`` as far apart from ``woman`` as ``king`` is from ``queen`` (roughly approximating the gender difference between the two). Similarly, the model learned to place ``king`` as far apart from ``man`` as ``queen`` is from ``woman`` (roughly approximating the difference of being a royal). We can visualize this concept when projected in two dimensions as shown below:

<img src="https://drive.google.com/uc?id=1TO3WS4-RKkL_AO---ZFAEzxcDLHjEMEu" width="40%">

From the figure above, it should be relatively clear that both arrows pointing up (blue) are approximately the same size, thus resulting in the equation below:

\begin{align}
    w_{king} - w_{man} &≃ w_{queen} - w_{woman}\\
    w_{king} - w_{man} &+ w_{woman} ≃ w_{queen}
\end{align}

We can observe how the word vectors also include information to relate countries with nationalities, months of the year, family relationships, etc.

In [ ]:
result = glove.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print('King - Man + Woman = ',result)
result = glove.most_similar(positive=['rome', 'france'], negative=['paris'], topn=1)
print('France - Paris + Rome = ',result)
result = glove.most_similar(positive=['english', 'france'], negative=['french'], topn=1)
print('France - french + english = ',result)
result = glove.most_similar(positive=['june', 'december'], negative=['november'], topn=1)
print('December - November + June = ',result)
result = glove.most_similar(positive=['sister', 'man'], negative=['woman'], topn=1)
print('Man - Woman + Sister = ',result)

King - Man + Woman =  [('queen', 0.8523604273796082)]
France - Paris + Rome =  [('italy', 0.8614554405212402)]
France - french + english =  [('england', 0.8093705177307129)]
December - November + June =  [('july', 0.9906677603721619)]
Man - Woman + Sister =  [('friend', 0.8550175428390503)]


In conclusion, word embeddings are indeed quite capable of capturing and maintaining the semantic relationship between different words.

### *3.3. Sentiment Analysis with pretrained GloVe embeddings*

##### **Exercise 6**

Analyze the vocabulary coverage of the GloVe embeddings on the IMBD dataset. How is it in comparison to the provided Word2Vec model trained on the IMBD corpus?

In [ ]:
# <SOL>

# </SOL>

##### **Exercise 7**

Obtain the embedding representation of the reviews based on GloVe's word vectors, train an SVM classifier with them and evaluate its accuracy. Use the same training and evaluation configuration as in Exercise 5.

In [ ]:
# <SOL>

# </SOL>

In [ ]:
del glove
gc.collect()

## *4. FastText*

### *4.1. Representation of OOV with ad-hoc vs pre-trained fastText model in Gensim*

We will be using [Gensim's implementation to learn word representations via fastText](https://radimrehurek.com/gensim/models/fasttext.html), which allows training word embeddings from a training corpus with the additional ability to obtain word vectors for out-of-vocabulary words.

Similar to what we did with Word2Vec, we can obtain the ad-hoc embeddings of our reviews based on the fastText model. Input must be given in the same format as Word2Vec, i.e., as a sequence of sentences, where each sentence is a list of words, that can be given as an iterator for efficient memory management.

Many of the construction parameters are common to the Word2Vec class:

* `vector_size` (embedding size): the number of dimensions for the vector representation. The default is 100.
* `window` (window size): the number of adjacent words that are considered to be in the same context as a target word. The default is 5.
* `min_count`: minimum number of times a word must appear in the corpus to be considered in the model.

Once trained, we can similarly save/load the model as we saw in Word2Vec.

##### **Exercise 8**

Train a FastText model named ``model_fasttext`` on the IMDB dataset. Use an embedding size of $200$, words must appear in at least $10$ documents to be kept in the model, and a window size of $5$ and random_state $42$. Use the class ``IterableSentence_fromfile`` above defined to create an iterator that yields one sentence (list of utf8 words) after another from the file ``"imdb_lemmas_clean.txt"``.

In [ ]:
%%time
from gensim.models import FastText

# <SOL>

# </SOL>

In [ ]:
# Save model
model_fasttext.save("model_fastText.model")

# Store just the words + their trained embeddings.
word_vectors = model_fasttext.wv
word_vectors.save("model_fastText.wordvectors")

# Load back with memory-mapping = read-only, shared across processes.
fastText_wv = KeyedVectors.load("model_fastText.wordvectors", mmap='r')

In [ ]:
del model_fasttext
gc.collect()

By representing the TSNE representation of the fastText embeddings we can see its better ability to group words with the same root (and similar meaning) as opposed to Word2Vec.

In [ ]:
tsne = TSNE(init='random')
embed_tsne = tsne.fit_transform(fastText_wv.vectors[:500,:])
fig, ax = plt.subplots(figsize=(16, 16))
for idx, word in enumerate((list(fastText_wv.key_to_index.keys())[:500])):
    plt.scatter(*embed_tsne[idx, :], color='steelblue')
    plt.annotate(word, (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)


plt.grid()

In [ ]:
del embed_tsne
gc.collect()

Another interesting thing to look at from the fastText model is its ability to group words with the same root (and similar meaning). This allows fastText's ability to obtain embeddings for out-of-dictionary (OOV) words, as opposed to Word2Vec:

In [ ]:
# Wor2Vec for word 'dichlorodiphenyltrichloroethane'
word = 'dichlorodiphenyltrichloroethane'
if (word in wv.key_to_index):
    print(wv[word])
else:
    print(f'The word {word} does not belong to the dictionary. Wor2Vec cannot assign an embedding.')

In [ ]:
# fastText constructs an embedding from the embeddings of the constituent subwords
if not (word in fastText_wv.key_to_index):
    print(f'The word {word} does not belong to the dictionary. The embedding constructed from the subwords is:\n')
    print(fastText_wv[word])
    print('\nAnd its closest neighbors are:')
    print(fastText_wv.most_similar(word))

The OOV embedding neighbours make little sense since the embedding dictionary was learned on a relatively tiny dataset. Let us now consider the usage of **pre-trained models**.

Since we want to keep fastText ability to obtain the embedding representation of OOV, using Gensim's KeyedVectors is not an option, as it only maintains a simple word (as key) to vector (as value) mapping. To solve this issue, we are instead going to be using the library [Compress-fastText
](https://pypi.org/project/compress-fasttext/), that allows us to load pretrained fastText models as instances of the class **CompressedFastTextKeyedVectors**, which does synthesize vectors for out-of-vocabulary words.


In [ ]:
try:
  import compress_fasttext
except ModuleNotFoundError:
  %pip install compress-fasttext
  import compress_fasttext

In [ ]:
fastTextPre = compress_fasttext.models.CompressedFastTextKeyedVectors.load(
    'https://github.com/avidale/compress-fasttext/releases/download/v0.0.4/cc.en.300.compressed.bin'
)

In [ ]:
if not (word in fastTextPre.key_to_index):
    print(f'The word {word} does not belong to the dictionary. The embedding constructed from the subwords is:\n')
    print(fastTextPre[word])
    print('\nAnd its closest neighbors are:')
    print(fastTextPre.most_similar(word))

As we can observe now, the OOV embedding neighbours do make sense.

### *4.2. Sentiment Analysis with ad-hoc and pre-trained fastText embeddings*

##### **Exercise 9**

Obtain the embedding representation of the reviews based on the **ad-hoc fastText's word vectors obtained from the IMDB dataset**, train an SVM classifier with them, and evaluate its performance. Use the same training and evaluation configuration as in Exercise 5.

In [ ]:
# <SOL>

# </SOL>

##### **Exercise 10**

Obtain the embedding representation of the reviews based on the **pre-trained fastText word vectors**, train an SVM classifier with them, and evaluate its performance. Use the same training and evaluation configuration as in Exercise 5.

In [ ]:
# <SOL>
# TODO: Obtain the embedding representation
# This part is going to print a lot of message, so it is better if you make the
# training of the classifier in the next cell
# </SOL>

In [ ]:
# <SOL>
# TODO: Train classifier with pre-trained fastText word vectors
# </SOL>

Additionally, the Compress-fastText library allows feeding fastText embeddings to some other model via the ``FastTextTransformer`` class, which makes use of the scikit-learn interface and **represents a text as the average of the embedding of its words**. Hence, we can directly use it for training our SVM on top of fastText as follows:

In [ ]:
import compress_fasttext
from sklearn.pipeline import make_pipeline
from compress_fasttext.feature_extraction import FastTextTransformer

corpus_train = [corpus[i] for i in id_train.tolist()]
corpus_test = [corpus[i] for i in id_test.tolist()]
y_train = Y[id_train]
y_test = Y[id_test]

parameters = {'kernel':('linear', 'rbf'), 'C':np.arange(1,11,2)}
classifier = make_pipeline(
    FastTextTransformer(model=fastTextPre),
    GridSearchCV(estimator=svm.SVC(), param_grid = parameters, cv = 2, n_jobs=-1)
).fit(corpus_train,y_train)
classifier.predict(corpus_test)
score_fastText_pipeline = classifier.score(corpus_test, y_test)
print("\n Accuracy with pre-trained FastText embeddings via FastTextTransformer :\n",score_fastText_pipeline)

In [ ]:
del fastTextPre
gc.collect()

## *5. ELMo*

The ELMo model consists of $L = 2$ biLSTM layers with $4,096$ units and $512$ dimension projections and a residual connection from the first to the second layer. It was trained on a really large corpus consisting of $5,5$ billion words. Since ELMo representations are character-based, they can handle OOV words. Additionally, it provides a contextual representation for each word which depends on the entire context in which it is used. You can find more details about both its implementation and pre-trained weights at [AllenNLP's ELMo site](https://allenai.org/allennlp/software/elmo).

ELMo embeddings are **deep** because their word representations combine all layers of the deep pre-trained neural network: they result from combining traditional word embeddings and the hidden states from the two biLSTMs. Because embeddings and hidden states both have $512 $dimensions, there is one $512$-dimension embedding and two $512$-dimension hidden states in each direction (one for each layer). That's $1536$ dimensions in each direction, which makes a total of $3072$ dimensions.

<img src="https://drive.google.com/uc?id=130IQhKGOS3d0WUZ9ameIimbjNSMqP_us" width="70%">

Because both LSTMs use identical inputs, the word embeddings are duplicated. The first two pieces of 512 dimensions are identical to the first 3072 dimensions.

<img src="https://drive.google.com/uc?id=1NhaDNcTHHYr44le7ofdR4pFa8UyC0omV" width="25%">

Since the classical word embeddings are context-independent, if we obtain the ELMo embeddings of two polysemic words they will share the same values in their first $1024$ dimensions.

### *5.1. ELMo embeddings in Flair*

We could train an ad-hoc ELMo model for our corpus, for example through the [AllenNLP's PyTorch-based NLP framework](https://guide.allennlp.org/), but for the sake of this tutorial we are going to retrieve the ELMo embeddings using the [Flair library](https://pypi.org/project/flair/). Flair is an NLP framework built on top of Pytorch that offers a text embedding library that provides word embeddings and document embeddings for models like ELMo, and BERT, as well as classical word embeddings like GloVe.

In [ ]:
%pip install allennlp==0.9.0
import torch
%pip install flair==0.4.3
import flair
%pip install overrides==3.1.0

<font color='red'>Be aware that you will probably need to restart the environment after executing the former cell. You do not need to execute the whole notebook again after it if you are just going to keep on working with this section. Still, make sure that you do execute the cells concerning the imports at the beginning, as well as the steps in **Section 1**; otherwise, you will not have access to the reviews, nor the labels. </font>

Flair data types have two objects, namely **sentence** and **token** objects. Sentences are lists of tokens that hold textual sentences. Then, every text sentence is a Sentence object that is created using the corresponding text.

In [ ]:
sentence1 = "My investment adviser at my local bank called to discuss my portfolio."
sentence2 = "Every day this sweet old lady sits on the same bank at my local park to feed the ducks."

sentences = [sentence1, sentence2]

In [ ]:
from flair.data import Sentence

flair_senteces = [Sentence(s) for s in sentences]

print(colored('\n============= Sentence =============', 'blue'))
print(sentences[0])
print(colored('\n============= Flair sentence =============', 'blue'))
print(flair_senteces[0])

Specific tokens can be retrieved through either the ``get_token()`` method or the ``tokens`` attribute. Please, note that the first method assumes indexing starts at one, while the attribute approach has the typical zero-based indexing.

In [ ]:
print(flair_senteces[0].get_token(7) == flair_senteces[0].tokens[6])
print(flair_senteces[0].get_token(6))

Now we can use these Sentence objects to retrieve their corresponding ELMo embeddings.

In [ ]:
from flair.embeddings import ELMoEmbeddings

elmo_embedding = ELMoEmbeddings()
elmo_embedding.embed(flair_senteces)

The example sentences we created above use the word ``bank`` in a different context. We can see how ELMo embeddings take this into account by assigning two different vectors depending on the context such a word was found in.

In [ ]:
token_bank1 = flair_senteces[0].tokens[6]
token_bank2 = flair_senteces[1].tokens[10]

print(colored('\n============= ELMo embedding of BANK in sentence 1 =============', 'blue'))
print(token_bank1.embedding)
print(colored('\n============= ELMo embedding of BANK in sentence 2 =============', 'blue'))
print(token_bank2.embedding)

If we calculate cosine similarity of the words through their embeddings, we can check that the two words are not so similar after all!

In [ ]:
import torch.nn as nn

similarity = nn.CosineSimilarity(dim=0, eps=1e-6)
print(similarity(token_bank1.embedding,token_bank2.embedding))

We can additionally check the two statements that we mentioned at the beginning of this ELMo section:


1.   Word embeddings are duplicated
2.   Polysemeic words share their first 1024 dimensions



In [ ]:
print(token_bank1.embedding[0] == token_bank1.embedding[512])
print(token_bank1.embedding[0])
print(colored('\n==========================', 'blue'))
print((token_bank1.embedding[:1024] == token_bank2.embedding[:1024]).all())

Focusing now on our IMBD reviews, one way of vectorizing them is to take the mean of the word embeddings within a review, as follows:

  For each sentence:
  - Generate word embedding for each word
  - Calculate the mean of the embeddings of each word to obtain the embedding of the sentence

In such a case, we need to provide the reviews to be embedded by Flair's ELMo as a list of sentences, rather than as a list of lists of tokens, which is what we have right now. So let's first transform our corpus.

In [ ]:
corpus = corpus_df['clean_review'].tolist()
corpus_join = [' '.join(i) for i in corpus]
print(colored('\n============= First review in corpus joint =============', 'blue'))
corpus_join[0]

The following cell implements the approach above described. Since it takes quite a long time to execute (around 45 minutes), we have saved for you the resulting NumPy array containing the embeddings. You can download them from [here](https://drive.google.com/file/d/1O0GN_V54C7Dl3R2M98Xr0Qi87dpXtLhn/view?usp=sharing). Save them into your Drive so you can unpickle them directly.

In [ ]:
%%time

################################################################################
# Uncomment the following code if you want to perform the calculation
################################################################################
'''
z = token_bank2.embedding.size()[0]
s = torch.zeros(0,z) # Tensor for storing sentence embeddings

for review in tqdm(corpus_join):
  w = torch.zeros(0,z) # Tensor for words
  sentence = Sentence(review)
  elmo_embedding.embed(sentence)
  for token in sentence:
    # Store WE of each word in a sentence
    w = torch.cat((w,token.embedding.view(-1,z)),0)
  # Store SE (mean of embeddings of all words)
  s = torch.cat((s, w.mean(dim = 0).view(-1, z)),0)
X = s.numpy()
'''

################################################################################
# Comment the following code if you uncommented the code above
################################################################################
X = unpickler("elmo_embeddings_imbd.pickle")

print(f"The dimensions of the ELMo embeddings are {X.shape[1]}")

### *5.2. Sentiment Analysis with Flair's ELMo embeddings*

##### **Exercise 11**

Train an SVM classifier with Flair's ELMo embeddings of the IMDB reviews and evaluate its performance. Use the same training and evaluation configuration as in Exercise 5.

In [ ]:
# <SOL>

# </SOL>